In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.weight_norm import weight_norm
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, matthews_corrcoef, confusion_matrix
from tqdm import tqdm
import os
import random

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
ESM_EMBED_DIM = 1920
STRUCT_EMBED_DIM = 128
RANDOM_SEED = 42

# Fixed configuration
config = {
    'batch_size': 128,
    'learning_rate': 1e-5,
    'num_epochs': 200,
    'patience': 5,
    'weight_decay': 1e-5,
    'hidden_dim': 128,
    'dropout': 0.4,
    'k': 3,
    'l1_lambda': 0
}

# Data configuration
BASE_PATH = '/mnt/e/Doctorale/Project/AMPBAN'
TRAIN_DATA = {
    'pos_embeddings': f'{BASE_PATH}/data/training_pos_aug.pth',
    'neg_embeddings': f'{BASE_PATH}/data/training_neg_aug.pth',
    'structure_embeddings': f'{BASE_PATH}/data/protein_egnn_embeddings.pt'
}
INDEPENDENT_DATASETS = {
    'MFA': {
        'sequence': f'{BASE_PATH}/independent_test_data/MFA_embeddings_by_id.pth',
        'structure': f'{BASE_PATH}/independent_test_data/MFA_independent_stru.pt'
    },
    'Xu': {
        'sequence': f'{BASE_PATH}/independent_test_data/xu_embeddings_by_id.pth',
        'structure': f'{BASE_PATH}/independent_test_data/xu_independent_structure.pt'
    },
    'Xiao': {
        'sequence': f'{BASE_PATH}/independent_test_data/Xiao_embeddings_by_id.pth',
        'structure': f'{BASE_PATH}/independent_test_data/xiao_independent_new_stru.pt'
    }
}

# Set seed for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(RANDOM_SEED)

In [5]:
# Model definitions
class FCNet(nn.Module):
    def __init__(self, dims, act='ReLU', dropout=0):
        super(FCNet, self).__init__()
        layers = []
        for i in range(len(dims) - 2):
            in_dim = dims[i]
            out_dim = dims[i + 1]
            if 0 < dropout:
                layers.append(nn.Dropout(dropout))
            layers.append(weight_norm(nn.Linear(in_dim, out_dim), dim=None))
            if '' != act:
                layers.append(getattr(nn, act)())
        if 0 < dropout:
            layers.append(nn.Dropout(dropout))
        layers.append(weight_norm(nn.Linear(dims[-2], dims[-1]), dim=None))
        if '' != act:
            layers.append(getattr(nn, act)())
        self.main = nn.Sequential(*layers)

    def forward(self, x):
        return self.main(x)

class BANLayer(nn.Module):
    def __init__(self, v_dim, q_dim, h_dim, h_out, act='ReLU', dropout=0.2, k=3):
        super(BANLayer, self).__init__()
        self.c = 32
        self.k = k
        self.v_dim = v_dim
        self.q_dim = q_dim
        self.h_dim = h_dim
        self.h_out = h_out
        self.v_net = FCNet([v_dim, h_dim * self.k], act=act, dropout=dropout)
        self.q_net = FCNet([q_dim, h_dim * self.k], act=act, dropout=dropout)
        if 1 < k:
            self.p_net = nn.AvgPool1d(self.k, stride=self.k)
        if h_out <= self.c:
            self.h_mat = nn.Parameter(torch.Tensor(1, h_out, 1, h_dim * self.k).normal_())
            self.h_bias = nn.Parameter(torch.Tensor(1, h_out, 1, 1).normal_())
        else:
            self.h_net = weight_norm(nn.Linear(h_dim * self.k, h_out), dim=None)
        self.bn = nn.BatchNorm1d(h_dim)

    def attention_pooling(self, v, q, att_map):
        expanded_att_map = att_map.unsqueeze(-1)
        expanded_v = v.unsqueeze(2)
        weighted_v = expanded_v * expanded_att_map
        fusion_logits = torch.sum(weighted_v * q.unsqueeze(1), dim=2)
        fusion_logits = torch.sum(fusion_logits, dim=1)
        if 1 < self.k:
            fusion_logits = fusion_logits.unsqueeze(1)
            fusion_logits = self.p_net(fusion_logits).squeeze(1) * self.k
        return fusion_logits

    def forward(self, v, q, softmax=False):
        v_num = v.size(1)
        q_num = q.size(1)
        if self.h_out <= self.c:
            v_ = self.v_net(v)
            q_ = self.q_net(q)
            att_maps = torch.einsum('ohik,bjk,blk->bojl', (self.h_mat.squeeze(0).squeeze(1), v_, q_)) + self.h_bias
            att_maps = att_maps.unsqueeze(1)
        else:
            v_ = self.v_net(v)
            q_ = self.q_net(q)
            logits = torch.einsum('bvk,bqk->bvqk', (v_, q_))
            att_maps = self.h_net(logits)
            att_maps = att_maps.transpose(1, 3).transpose(2, 3)
        if softmax:
            p = nn.functional.softmax(att_maps.view(-1, self.h_out, v_num * q_num), 2)
            att_maps = p.view(-1, self.h_out, v_num, q_num)
        logits = self.attention_pooling(v_, q_, att_maps[:, 0, :, :])
        for i in range(1, self.h_out):
            logits_i = self.attention_pooling(v_, q_, att_maps[:, i, :, :])
            logits += logits_i
        logits = self.bn(logits)
        return logits, att_maps

class AMPBANModel(nn.Module):
    def __init__(self, config):
        super(AMPBANModel, self).__init__()
        self.ban = BANLayer(v_dim=STRUCT_EMBED_DIM, q_dim=ESM_EMBED_DIM, h_dim=config['hidden_dim'], h_out=config['hidden_dim'], dropout=config['dropout'])
        self.classifier = nn.Sequential(
            nn.Linear(ESM_EMBED_DIM + STRUCT_EMBED_DIM + config['hidden_dim'], config['hidden_dim']),  # 1920 + 128 + 128
            nn.BatchNorm1d(config['hidden_dim']),
            nn.ReLU(),
            nn.Dropout(config['dropout']),
            nn.Linear(config['hidden_dim'], 1),
            nn.Sigmoid()
        )

    def forward(self, sequence, structure):
        seq_features = sequence
        struct_features = structure
        joint_features, _ = self.ban(struct_features.unsqueeze(1), seq_features.unsqueeze(1))
        combined = torch.cat([sequence, structure, joint_features], dim=1)
        return self.classifier(combined).squeeze()


In [6]:
# Dataset class with enhanced debugging and error handling
class AMPDataset(Dataset):
    def __init__(self, sequence_embeddings_path=None, structure_embeddings_path=None, pos_embeddings_path=None, neg_embeddings_path=None, is_train=False):
        if is_train:
            if not (pos_embeddings_path and neg_embeddings_path and structure_embeddings_path):
                raise ValueError("Must provide pos_embeddings_path, neg_embeddings_path, and structure_embeddings_path for training.")
            if not all(os.path.exists(p) for p in [pos_embeddings_path, neg_embeddings_path, structure_embeddings_path]):
                raise FileNotFoundError(f"One or more embedding files not found: pos={pos_embeddings_path}, neg={neg_embeddings_path}, struct={structure_embeddings_path}")
            pos_embeddings_dict = torch.load(pos_embeddings_path, weights_only=True)
            neg_embeddings_dict = torch.load(neg_embeddings_path, weights_only=True)
            self.structure_embeddings_dict = torch.load(structure_embeddings_path, weights_only=True)
            print(f"Loaded pos_embeddings: {len(pos_embeddings_dict)} entries, Sample keys: {list(pos_embeddings_dict.keys())[:5]}")
            print(f"Loaded neg_embeddings: {len(neg_embeddings_dict)} entries, Sample keys: {list(neg_embeddings_dict.keys())[:5]}")
            print(f"Loaded structure_embeddings: {len(self.structure_embeddings_dict)} entries, Sample keys: {list(self.structure_embeddings_dict.keys())[:5]}")
            pos_ids = list(pos_embeddings_dict.keys())
            neg_ids = list(neg_embeddings_dict.keys())
            self.sequence_ids = pos_ids + neg_ids
            if not self.sequence_ids:
                raise ValueError("Sequence IDs list is empty. Both positive and negative embeddings are empty.")
            missing_ids = [sid for sid in self.sequence_ids if sid not in self.structure_embeddings_dict]
            if missing_ids:
                raise KeyError(f"Structure embeddings missing for {len(missing_ids)} sequence IDs: {missing_ids[:5]}...")
            pos_embeddings = [pos_embeddings_dict[sid] for sid in pos_ids]
            neg_embeddings = [neg_embeddings_dict[sid] for sid in neg_ids]
            if not pos_embeddings:
                raise ValueError("Positive embeddings list is empty.")
            if not neg_embeddings:
                raise ValueError("Negative embeddings list is empty.")
            all_embeddings = pos_embeddings + neg_embeddings
            if not all_embeddings:
                raise ValueError("Combined embeddings list is empty.")
            self.sequence_embeddings = torch.stack(all_embeddings)
            self.labels = torch.cat([torch.ones(len(pos_embeddings)), torch.zeros(len(neg_embeddings))])
        else:
            if not (sequence_embeddings_path and structure_embeddings_path):
                raise ValueError("Must provide sequence_embeddings_path and structure_embeddings_path for testing.")
            if not os.path.exists(sequence_embeddings_path) or not os.path.exists(structure_embeddings_path):
                raise FileNotFoundError(f"One or more embedding files not found: seq={sequence_embeddings_path}, struct={structure_embeddings_path}")
            self.sequence_embeddings_dict = torch.load(sequence_embeddings_path, weights_only=True)
            self.structure_embeddings_dict = torch.load(structure_embeddings_path, weights_only=True)
            self.sequence_ids = list(self.sequence_embeddings_dict.keys())
            print(f"Loaded test sequence_embeddings: {len(self.sequence_embeddings_dict)} entries, Sample keys: {list(self.sequence_embeddings_dict.keys())[:5]}")
            print(f"Loaded test structure_embeddings: {len(self.structure_embeddings_dict)} entries, Sample keys: {list(self.structure_embeddings_dict.keys())[:5]}")
            if not self.sequence_ids:
                raise ValueError("Test sequence embeddings dictionary is empty.")
            self.labels = torch.tensor([1 if sid.endswith('_1') else 0 for sid in self.sequence_ids])
            if not all(sid.endswith(('_1', '_0')) for sid in self.sequence_ids):
                invalid_ids = [sid for sid in self.sequence_ids if not sid.endswith(('_1', '_0'))]
                raise ValueError(f"Some test sequence IDs do not end with '_1' or '_0': {invalid_ids[:5]}...")
            missing_ids = [sid for sid in self.sequence_ids if sid not in self.structure_embeddings_dict]
            if missing_ids:
                raise KeyError(f"Structure embeddings missing for {len(missing_ids)} sequence IDs: {missing_ids[:5]}...")
            test_embeddings = [self.sequence_embeddings_dict[sid] for sid in self.sequence_ids]
            if not test_embeddings:
                raise ValueError("Test sequence embeddings list is empty.")
            self.sequence_embeddings = torch.stack(test_embeddings)
        self.is_train = is_train

    def __len__(self):
        return len(self.sequence_ids)

    def __getitem__(self, idx):
        seq_id = self.sequence_ids[idx]
        sequence_embedding = self.sequence_embeddings[idx]
        structure_embedding = self.structure_embeddings_dict[seq_id]
        return {
            'sequence': sequence_embedding,
            'structure': structure_embedding,
            'label': self.labels[idx],
            'seq_id': seq_id
        }

# Training function with early stopping
def train_model(model, train_loader, val_loader, config, device):
    model = model.to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config.get('weight_decay', 0))
    l1_lambda = config.get('l1_lambda', 0)
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    train_losses = []
    val_losses = []

    for epoch in range(config['num_epochs']):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            sequences = batch['sequence'].to(device)
            structures = batch['structure'].to(device)
            labels = batch['label'].float().to(device)
            outputs = model(sequences, structures)
            loss = criterion(outputs, labels)
            if l1_lambda > 0:
                l1_penalty = torch.tensor(0., requires_grad=True).to(device)
                for name, param in model.named_parameters():
                    if 'bias' not in name and param.requires_grad:
                        l1_penalty = l1_penalty + torch.abs(param).sum()
                loss = loss + l1_lambda * l1_penalty
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * sequences.size(0)
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        val_metrics = evaluate_model(model, val_loader, criterion, device)
        val_loss = val_metrics['loss']
        val_losses.append(val_loss)
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val AUC: {val_metrics['auc']:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= config['patience']:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                break

    plot_loss_curves(train_losses, val_losses)
    return best_model_state, val_metrics, train_losses, val_losses

# Evaluation function
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_labels = []
    all_preds = []
    all_probs = []
    with torch.no_grad():
        for batch in data_loader:
            sequences = batch['sequence'].to(device)
            structures = batch['structure'].to(device)
            labels = batch['label'].float().to(device)
            outputs = model(sequences, structures)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * sequences.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(outputs.cpu().numpy())
            all_preds.extend((outputs > 0.5).float().cpu().numpy())
    unique_labels = set(all_labels)

    if len(unique_labels) <= 1:
        print(f"Warning: Validation set has only one class ({unique_labels}). AUC set to 0.0.")
        auc = 0.0
    else:
        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        spec = tn / (tn + fp) if (tn + fp) else 0.0
        auc = roc_auc_score(all_labels, all_probs)
    metrics = {
        'loss': total_loss / len(data_loader.dataset) if len(data_loader.dataset) > 0 else 0.0,
        'accuracy': accuracy_score(all_labels, all_preds),
        'mcc': matthews_corrcoef(all_labels, all_preds),
        'sensitivity': recall_score(all_labels, all_preds, zero_division=0),
        'specificity': spec,
        'precision': precision_score(all_labels, all_preds, zero_division=0),
        'f1': f1_score(all_labels, all_preds, zero_division=0),
        'auc': auc
    }
    return metrics

# Plotting function
def plot_loss_curves(train_losses, val_losses):
    if not train_losses or not val_losses:
        print("No loss data to plot. Skipping.")
        return
    import matplotlib.pyplot as plt
    epochs = range(1, min(len(train_losses), len(val_losses)) + 1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses[:len(epochs)], label='Training Loss')
    plt.plot(epochs, val_losses[:len(epochs)], label='Validation Loss')
    plt.title('Training and Validation Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig('loss_curve.png')
    plt.close()

def main():
    # Load training dataset
    try:
        train_dataset = AMPDataset(
            pos_embeddings_path=TRAIN_DATA['pos_embeddings'],
            neg_embeddings_path=TRAIN_DATA['neg_embeddings'],
            structure_embeddings_path=TRAIN_DATA['structure_embeddings'],
            is_train=True
        )
    except Exception as e:
        print(f"Error loading training dataset: {e}")
        return

    # Use StratifiedKFold for 5-fold cross-validation
    labels = train_dataset.labels.numpy()
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    best_val_auc = -1
    best_val_loss = float('inf')
    best_model_state = None

    for fold, (train_idx, val_idx) in enumerate(kf.split(np.zeros(len(labels)), labels)):
        print(f"\n--- Fold {fold+1}/5 ---")
        train_subset = Subset(train_dataset, train_idx)
        val_subset = Subset(train_dataset, val_idx)
        train_labels = [train_dataset.labels[i].item() for i in train_idx]
        val_labels = [train_dataset.labels[i].item() for i in val_idx]
        print(f"Training fold class distribution: {np.bincount(train_labels)}")
        print(f"Validation fold class distribution: {np.bincount(val_labels)}")
        train_loader = DataLoader(train_subset, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=config['batch_size'])
        model = AMPBANModel(config).to(device)
        model_state, val_metrics, _, _ = train_model(model, train_loader, val_loader, config, device)
        print(f"Fold {fold+1} Validation Metrics: Accuracy: {val_metrics['accuracy']:.4f}, "
              f"MCC: {val_metrics['mcc']:.4f}, Sensitivity: {val_metrics['sensitivity']:.4f}, "
              f"Precision: {val_metrics['precision']:.4f}, F1: {val_metrics['f1']:.4f}, "
              f"AUC: {val_metrics['auc']:.4f}, specificity: {val_metrics['specificity']:.4f}")

        if not np.isnan(val_metrics['auc']) and val_metrics['auc'] > best_val_auc:
            best_val_auc = val_metrics['auc']
            best_val_loss = val_metrics['loss']
            best_model_state = model_state
        elif np.isnan(val_metrics['auc']) and val_metrics['loss'] < best_val_loss:
            print(f"Warning: AUC is NaN for fold {fold+1}. Using validation loss as fallback metric.")
            best_val_auc = val_metrics['auc']
            best_val_loss = val_metrics['loss']
            best_model_state = model_state

    # Train and save best model on full training data
    print("\n--- Training Best Model on Full Training Data ---")
    print("Best Average Validation AUC:", best_val_auc if not np.isnan(best_val_auc) else "NaN (used loss as fallback)")
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    model = AMPBANModel(config).to(device)
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    model_state, _, _, _ = train_model(model, train_loader, train_loader, config, device)
    torch.save(model_state, 'best_amp_ban_model.pth')
    print("Saved best model to 'best_amp_ban_model.pth'")

    # Evaluate on independent test datasets
    for dataset_name, paths in INDEPENDENT_DATASETS.items():
        print(f"\n--- Evaluating Best Model on {dataset_name} Independent Test Data ---")
        try:
            test_dataset = AMPDataset(
                sequence_embeddings_path=paths['sequence'],
                structure_embeddings_path=paths['structure'],
                is_train=False
            )
            test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])
            model.load_state_dict(model_state)
            criterion = nn.BCELoss()
            test_metrics = evaluate_model(model, test_loader, criterion, device)
            print(f"{dataset_name} Test Metrics: Accuracy: {test_metrics['accuracy']:.4f}, "
                  f"MCC: {test_metrics['mcc']:.4f}, Sensitivity: {test_metrics['sensitivity']:.4f}, "
                  f"Precision: {test_metrics['precision']:.4f}, F1: {test_metrics['f1']:.4f}, "
                  f"AUC: {test_metrics['auc']:.4f}")
        except Exception as e:
            print(f"Error evaluating {dataset_name} test data: {e}. Skipping this dataset.")

if __name__ == '__main__':
    main()

Loaded pos_embeddings: 30971 entries, Sample keys: ['76', '77', '78', '79', '82']
Loaded neg_embeddings: 38675 entries, Sample keys: ['sp|A0A068B6Q6|CA18_CONBE', 'sp|A0A0B4J1N3|GP15L_MOUSE', 'sp|A0A0B4J2F0|PIOS1_HUMAN', 'sp|A0A0C5B5G6|MOTSC_HUMAN', 'sp|A0A0N7CSQ4|TX41A_SCOMU']
Loaded structure_embeddings: 69646 entries, Sample keys: ['100', '101', '102', '103', '104']

--- Fold 1/5 ---
Training fold class distribution: [30940 24776]
Validation fold class distribution: [7735 6195]


/tmp/ipykernel_71814/2931172422.py:208: DeprecationWarning: Non-integer input passed to bincount. In a future version of NumPy, this will be an error. (Deprecated NumPy 2.1)
  print(f"Training fold class distribution: {np.bincount(train_labels)}")
/tmp/ipykernel_71814/2931172422.py:209: DeprecationWarning: Non-integer input passed to bincount. In a future version of NumPy, this will be an error. (Deprecated NumPy 2.1)
  print(f"Validation fold class distribution: {np.bincount(val_labels)}")
/home/baiwenhui/miniconda3/envs/deepamp/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 436/436 [00:30<00:00, 14.26it/s]


Epoch 1: Train Loss: 0.4123, Val Loss: 0.3298, Val AUC: 0.9362


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████| 436/436 [00:26<00:00, 16.53it/s]


Epoch 2: Train Loss: 0.3195, Val Loss: 0.2958, Val AUC: 0.9483


...

Epoch 74: Train Loss: 0.1345, Val Loss: 0.1214, Val AUC: 0.9912


Epoch 75: 100%|██████████████████████████████████████████████████████████████████████| 545/545 [00:33<00:00, 16.43it/s]


Epoch 75: Train Loss: 0.1352, Val Loss: 0.1214, Val AUC: 0.9914


Epoch 76: 100%|██████████████████████████████████████████████████████████████████████| 545/545 [00:32<00:00, 16.60it/s]


Epoch 76: Train Loss: 0.1339, Val Loss: 0.1227, Val AUC: 0.9912
Early stopping triggered after 76 epochs.
Saved best model to 'best_amp_ban_model.pth'

--- Evaluating Best Model on MFA Independent Test Data ---
Loaded test sequence_embeddings: 1221 entries, Sample keys: ['AMPs001_1', 'AMPs002_1', 'AMPs003_1', 'AMPs004_1', 'AMPs005_1']
Loaded test structure_embeddings: 1221 entries, Sample keys: ['AMPs001_1', 'AMPs002_1', 'AMPs003_1', 'AMPs004_1', 'AMPs005_1']
MFA Test Metrics: Accuracy: 0.9525, MCC: 0.9050, Sensitivity: 0.9496, Precision: 0.9558, F1: 0.9527, AUC: 0.9878

--- Evaluating Best Model on Xu Independent Test Data ---
Loaded test sequence_embeddings: 3072 entries, Sample keys: ['ALL000004_1', 'ALL000015_1', 'ALL000016_1', 'ALL000022_1', 'ALL000030_1']
Loaded test structure_embeddings: 3072 entries, Sample keys: ['ALL000004_1', 'ALL000015_1', 'ALL000016_1', 'ALL000022_1', 'ALL000030_1']
Xu Test Metrics: Accuracy: 0.7077, MCC: 0.4691, Sensitivity: 0.4753, Precision: 0.8881, F1:

In [6]:
# Now
Fold 1 Validation Metrics: Accuracy: 0.9301, MCC: 0.8597, Sensitivity: 0.9437, Precision: 0.9034, F1: 0.9231, AUC: 0.9776, specificity: 0.9192
Fold 2 Validation Metrics: Accuracy: 0.9281, MCC: 0.8564, Sensitivity: 0.9491, Precision: 0.8954, F1: 0.9215, AUC: 0.9792, specificity: 0.9112
Fold 3 Validation Metrics: Accuracy: 0.9305, MCC: 0.8602, Sensitivity: 0.9399, Precision: 0.9071, F1: 0.9232, AUC: 0.9779, specificity: 0.9229
Fold 4 Validation Metrics: Accuracy: 0.9333, MCC: 0.8671, Sensitivity: 0.9558, Precision: 0.9004, F1: 0.9272, AUC: 0.9797, specificity: 0.9153
Fold 5 Validation Metrics: Accuracy: 0.9300, MCC: 0.8595, Sensitivity: 0.9428, Precision: 0.9039, F1: 0.9230, AUC: 0.9780, specificity: 0.9197

SyntaxError: invalid syntax (1878724523.py, line 1)